<a href="https://colab.research.google.com/github/HuyKhoi-code/NLP/blob/main/POS_TAGGING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT CÁC THƯ VIỆN CẦN THIẾT

In [ ]:
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

TIỀN XỬ LÝ GOLD DATA

In [ ]:
# ĐỌC FILE 
f = open('/content/Gold_data.txt')
data = f.read()

In [ ]:
# TÁCH TỪ (WORD SEGMENTATION)
data = data.split('\n')
for i in range(len(data)):
  data[i]=data[i].split()
for i in data:
  for y in range(len(i)):
    i[y]=tuple(i[y].split('/'))

In [ ]:
data_words = []
all = []
for sent in data:
    for words_tag in sent:
        data_words.append(words_tag)
for tup in data_words:
    all.append(tup[0])
print (all)
print (len(all))

['In', 'ancient', 'Greek', ',', 'there', 'was', 'a', 'king', 'named', 'Midas', '.', 'He', 'had', 'a', 'lot', 'of', 'gold', 'and', 'everything', 'he', 'needed', '.', 'He', 'also', 'had', 'a', 'beautiful', 'daughter', '.', 'Midas', 'loved', 'his', 'gold', 'very', 'much', ',', 'but', 'he', 'loved', 'his', 'daughter', 'more', 'than', 'his', 'riches', '.', 'One', 'day', ',', 'a', 'Satyr', 'named', 'Silenus', 'got', 'drunk', 'and', 'passed', 'out', 'in', 'Midas', "'s", 'rose', 'garden', '.', 'Believing', 'that', 'Satyrs', 'always', 'bring', 'good', 'luck', ',', 'Midas', 'lets', 'Silenus', 'rest', 'in', 'his', 'palace', 'until', 'he', 'is', 'sober', ',', 'against', 'the', 'wishes', 'of', 'his', 'wife', 'and', 'daughter', '.', 'Silenus', 'is', 'a', 'friend', 'of', 'Dionysus', ',', 'the', 'god', 'of', 'wine', 'and', 'celebration', '.', 'Upon', 'learning', 'Midas', '’s', 'kindness', 'towards', 'his', 'friend', ',', 'Dionysus', 'decided', 'to', 'reward', 'the', 'keg', '.', 'When', 'asked', 'to', 

In [ ]:
# CHIA TRAIN, TEST SET
train_set, test_set = train_test_split(data,test_size=0.2, random_state = 0)

# SỐ CÂU SỬ DỤNG CHO TRAIN SET
print(len(train_set))

# SỐ CÂU SỬ DỤNG CHO TEST SET
print(len(test_set))

36
9


In [ ]:
train_words = []
for sent in train_set:
    for word_tag in sent:
        train_words.append(word_tag)

# SỐ TỪ SỬ DỤNG CHO TRAIN SET
len(train_words)

481

In [ ]:
word = []
tag = []
for tup in train_words:
    word.append(tup[0])
    tag.append(tup[1])
print(len(word))
print(len(tag))

481
481


In [ ]:
# TẠO BỘ VOCABULARY
vocab = set(word)
len(vocab)

278

In [ ]:
T = set(tag)
len(T)

33

XÂY DỰNG MÔ HÌNH HIDDEN MARKOV

In [ ]:
#emision prob
def word_given_tag(word, tag, train_bag = train_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [ ]:
# transtition prob
def t2_given_t1(t2, t1, train_bag = train_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [ ]:
# Xây dựng transition matrix
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = (t2_given_t1(t2, t1)[0]+1)/(t2_given_t1(t2, t1)[1]+len(T))

In [ ]:
tags_matrix.shape

(33, 33)

In [ ]:
tags_matrix

array([[0.02666667, 0.02666667, 0.02666667, ..., 0.09333333, 0.01333333,
        0.01333333],
       [0.02857143, 0.02857143, 0.02857143, ..., 0.02857143, 0.02857143,
        0.02857143],
       [0.02702703, 0.02702703, 0.02702703, ..., 0.02702703, 0.02702703,
        0.02702703],
       ...,
       [0.01538462, 0.01538462, 0.01538462, ..., 0.01538462, 0.03076923,
        0.01538462],
       [0.02857143, 0.02857143, 0.02857143, ..., 0.02857143, 0.02857143,
        0.02857143],
       [0.01886792, 0.01886792, 0.03773585, ..., 0.11320755, 0.01886792,
        0.03773585]], dtype=float32)

In [ ]:
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,VB,WDT,"""",TO,WRB,.,VBN,VBG,JJS,VBP,NN,PDT,",",CC,VGB,IN,JJR,PP$,NNP,NNS,CD,RP,VBZ,RBS,POS,WP,RB,MD,PP,JJ,DT,NNPS,VBD
VB,0.026667,0.026667,0.026667,0.013333,0.040000,0.026667,0.026667,0.066667,0.013333,0.013333,0.053333,0.013333,0.026667,0.026667,0.013333,0.053333,0.013333,0.040000,0.026667,0.026667,0.040000,0.026667,0.026667,0.013333,0.013333,0.013333,0.053333,0.013333,0.026667,0.080000,0.093333,0.013333,0.013333
WDT,0.028571,0.028571,0.028571,0.028571,0.028571,0.057143,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.057143,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571
"""",0.027027,0.027027,0.027027,0.027027,0.027027,0.054054,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.108108,0.027027,0.027027,0.027027,0.027027
TO,0.265306,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.081633,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.040816,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408,0.020408
WRB,0.025641,0.025641,0.025641,0.051282,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.025641,0.076923,0.076923,0.025641,0.025641,0.051282
.,0.014286,0.014286,0.028571,0.014286,0.042857,0.014286,0.014286,0.028571,0.014286,0.014286,0.028571,0.014286,0.014286,0.014286,0.014286,0.100000,0.014286,0.014286,0.171429,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.028571,0.028571,0.028571,0.100000,0.014286,0.085714,0.014286,0.014286
VBN,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.085714,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571
VBG,0.023256,0.023256,0.023256,0.046512,0.023256,0.046512,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.023256,0.023256,0.093023,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.069767,0.023256,0.023256
JJS,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.058824,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412
VBP,0.048780,0.024390,0.024390,0.048780,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.073171,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.024390,0.048780,0.048780,0.073171,0.024390,0.024390


THUẬT TOÁN VITERBI

In [ ]:
def Viterbi(words, train_bag = train_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # TÍNH XÁC SUẤT 
            emission_p =( word_given_tag(words[key], tag)[0]+1)/(word_given_tag(words[key], tag)[1] + len(vocab))
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # LẤY NHÃN CÓ XÁC SUẤT CAO NHẤT 
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

DỰ ĐOÁN TRÊN TEST SET

In [ ]:
test_tuple = [tup for sent in test_set for tup in sent]

test_words = [tup[0] for sent in test_set for tup in sent]

In [ ]:
# GÁN NHÃN BẰNG VITERBI
test_tag = Viterbi(test_words)

In [ ]:
# TAG BỊ GÁN SAI 
for i in range (len(test_tag)):
    if (test_tag[i] != test_tuple[i]):
      print (test_tag[i], test_tuple[i])

('don’t', 'VBP') ('don’t', 'VB')
('that', 'DT') ('that', 'WDT')
('or', 'DT') ('or', 'CC')
('because', 'NN') ('because', 'CC')
('2020', 'DT') ('2020', 'CD')
('kilometres', 'NN') ('kilometres', 'NNS')
('all', 'DT') ('all', 'PDT')
('that', 'NN') ('that', 'WDT')
('exceptional', '.') ('exceptional', 'JJ')
('When', 'NNP') ('When', 'WRP')
('people', 'NNP') ('people', 'NN')
('go', 'NNP') ('go', 'VB')
('grocery', 'NNP') ('grocery', 'NN')
('shopping', 'NNP') ('shopping', 'NN')
('walk', 'VBP') ('walk', 'VB')
('That', 'NNP') ('That', 'DT')
('means', 'NNP') ('means', 'VBZ')
('any', 'NN') ('any', 'DT')
('word', '.') ('word', 'NN')
('definition', 'PP') ('definition', 'NN')
('useful', 'PP') ('useful', 'JJ')
('examples', 'VBD') ('examples', 'NNS')
('One', 'NNP') ('One', 'LS')
('Satyr', 'NN') ('Satyr', 'NNP')
('named', '.') ('named', 'VBD')
('got', 'NNP') ('got', 'VBD')
('drunk', 'NNP') ('drunk', 'JJ')
('passed', 'PP') ('passed', 'VBD')
('out', 'VBD') ('out', 'RB')
('rose', 'JJ') ('rose', 'NN')
('says',

TÍNH ACCURACY

In [ ]:
check = [i for i, j in zip(test_tag, test_tuple) if i == j] 
accuracy = len(check)/len(test_tag)

In [ ]:
accuracy

0.5

In [ ]:
print("F1 score on Test Data ")
print(f1_score(test_tuple[1], test_tag[1], average='weighted'))

F1 score on Test Data 
0.5


TEST THỬ NGHIỆM TRÊN 1 CÂU 

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
## TEST ON A SENTENCE
sentence_test = 'I gave Mike a kiss.'
words = word_tokenize(sentence_test)
print(words)
start = time.time()
tagged_sent = Viterbi(words)
end = time.time()
difference = end-start

['I', 'gave', 'Mike', 'a', 'kiss', '.']


In [ ]:
print(tagged_sent)
print(difference)

[('I', 'PP'), ('gave', 'VBD'), ('Mike', 'DT'), ('a', 'DT'), ('kiss', 'NN'), ('.', '.')]
0.019585847854614258


CRF MODEL

In [ ]:
pip install sklearn_crfsuite 

In [ ]:
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
from sklearn_crfsuite import CRF

In [ ]:
def features(sentence,index):
    ### sentence is of the form [w1,w2,w3,..], index is the position of the word in the sentence
    return {
        'is_first_capital':int(sentence[index][0].isupper()),
        'is_first_word': int(index==0),
        'is_last_word':int(index==len(sentence)-1),
        'is_complete_capital': int(sentence[index].upper()==sentence[index]),
        'prev_word':'' if index==0 else sentence[index-1],
        'next_word':'' if index==len(sentence)-1 else sentence[index+1],
        'is_numeric':int(sentence[index].isdigit()),
        'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',sentence[index])))),
        'prefix_1':sentence[index][0],
        'prefix_2': sentence[index][:2],
        'prefix_3':sentence[index][:3],
        'prefix_4':sentence[index][:4],
        'suffix_1':sentence[index][-1],
        'suffix_2':sentence[index][-2:],
        'suffix_3':sentence[index][-3:],
        'suffix_4':sentence[index][-4:],
        'word_has_hyphen': 1 if '-' in sentence[index] else 0,
        'capitals_inside': 1 if sentence[index][1:].lower() != sentence[index][1:] else 0,
         }
def untag(sentence):
    return [word for word,tag in sentence]
 
 
def prepareData(tagged_sentences):
    X,y=[],[]
    for sentences in tagged_sentences:
        X.append([features(untag(sentences), index) for index in range(len(sentences))])
        y.append([tag for word,tag in sentences])
    return X,y
X_train,y_train=prepareData(train_set)
X_test,y_test=prepareData(test_set)

In [ ]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.01, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [ ]:
y_pred=crf.predict(X_test)

In [ ]:
count = 0
for i in range (len(y_pred)):
  for j in range (len(y_pred[i])):
    if (y_pred[i][j] == y_test[i][j]):
      count+=1

In [ ]:
crf_accu = count/len(test_tagged_words)
print (crf_accu)

0.7131147540983607
